In [1]:
# ÖAI Collection Checker
# Returns the number of folders/files and the total size
# Returns all file extensions
# Returns filenames with special character
# @Author Karl Burkhart

import os
import hashlib

show_files = True
show_extensions = True
show_wrong_files = False
show_duplicates = False
create_file = False

# Liste an Sonderzeichen, die überprüft werden sollen
# ToDo Punkt in Namen check
files_special_chars=['Ä','ä','Ü','ü','Ö','ö','+',' ','!','@','.','#','\'']

# Ordner der rekursiv untersucht wird 
path = 'H:'

# name des metadatenfiles
filename_metadata = '#_metadata.txt'

files = []
files_list = []
files_dup = []
files_dic ={}
directories=[]
size = 0
files_special=[]
directories_special=[]
ausgabe = ''

# hashed eine Datei 
def hashfile(filename):
    with open(filename, 'rb') as f:
        hasher = hashlib.sha1()
        while True:
            data = f.read(1024)
            if not data:
                break
            hasher.update(data)
        return hasher.hexdigest()

# geht den Ordner durch und vergleich den Hashwert der darin enthaltenen Dateien ob Duplikate enthalten sind
def find_duplicates(dirname):
    global ausgabe
    files = os.listdir(dirname)
    if len(files) < 2:
        return
    hashes = {}
    for filename in files:
        path = os.path.join(dirname, filename)
        if not os.path.isfile(path):
            continue
        file_hash = hashfile(path)
        if file_hash not in hashes:
            hashes[file_hash] = path
        else:
            ausgabe+=(f'Duplikat gefunden: {path} und {hashes[file_hash]} \n')

# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    directories.append(d)
    
    # Überprüft ob Sonderzeichen in Ordnernamen enthalten sind
    for dic in d:
        files_dup.append(os.path.join(r,dic))
        for s in files_special_chars:
            if s in dic:
                directories_special.append(os.path.join(r,dic))
    
    for file in f:
        file_orig = os.path.join(r, file)
        files.append(file_orig)
        
        # Berechnet die Größe aller Dateien
        size += os.path.getsize(file_orig)
        
        filename, file_extension = os.path.splitext(file)
        files_list.append(filename+file_extension)

        # Überprüft ob Sonderzeichen aus Liste in Filename enthalten ist
        for s in files_special_chars:
            if s in filename:
                files_special.append(file_orig)
                
        # Legt neuen Eintrag für die Extensions an oder zählt die Anzahl um eines rauf
        if file_extension in files_dic:
            files_dic[file_extension]+=1
        else:
            files_dic[file_extension]=1

            
# Sortiert nach Anzahl der Extensions    
files_dic = dict(reversed(sorted(files_dic.items(), key=lambda item: item[1])))            
            
# Modul 1 - Anzeige der Anzahl von Dateien und Ordner sowie deren Größe
if show_files == True: 
    ausgabe += (f"Anzahl Dateien: {len(files)}\n")
    ausgabe +=(f"Anzahl Ordner: {len(directories)-1}\n")
    ausgabe +=(f"Größe des Ordners: {float(round(size/1000000))} MB\n")

# Modul 2 - Anzeige aller Fileerweiterungen 
if show_extensions == True:
    ausgabe +=(f"\nExtensions und deren Anzahl:\n")
    for key, value in files_dic.items():
        if key == '':
            ausgabe +=(f"Ohne Extension: {value}\n")
        else: 
            ausgabe +=(f"{key} {value}\n")

# Modul 3 - Anzeige aller Datei- und Ordnernamen mit Sonderzeichen
if show_wrong_files == True:   
    ausgabe +=(f"\nFiles mit Sonderzeichen (Anzahl {len(files_special)}):\n")
    for wrong_file in files_special:
        ausgabe +=(f"{wrong_file}\n")
    ausgabe +=(f"\nOrdner mit Sonderzeichen (Anzahl {len(directories_special)}):\n")
    for wrong_dic in directories_special:
        ausgabe +=(f"{wrong_dic}\n")
        
# Modul 4 - Anzeige aller gefundenen Duplikate in einem Ordner
if show_duplicates == True:
    ausgabe +=(f"\nDuplikate: \n")
    files_dup.append(path)
    for i in files_dup:
        find_duplicates(i)
    
print(ausgabe)
        
if create_file == True:         
    name = os.path.join(path, filename_metadata)
    with open(name, 'w', encoding='utf-8') as file_object:
        file_object.write(ausgabe)
        print("\n--Metadatenfile erstellt--")
        

      
        

Anzahl Dateien: 475
Anzahl Ordner: 36
Größe des Ordners: 2422.0 MB

Extensions und deren Anzahl:
.JPG 151
.csv 99
.png 86
.mdb 18
.zip 17
.siard 17
.PNG 16
.db 15
.jpg 14
.py 10
.pdf 9
.accdb 5
Ohne Extension: 5
.txt 3
.doc 2
.jpeg 2
.oc3 1
.psx 1
.psd 1
.tif 1
.mtl 1
.blend 1


--Metadatenfile erstellt--
